# STOR 120 - Homework 04: Town of Chapel Hill GIS and Analytics

This week, we'll bring together much of the content that we've learned so far in the course to investigate some local data.

**Throughout this homework and all future ones, please be sure to not re-assign variables throughout the notebook!** For example, if you use `max_temperature` in your answer to one question, do not reassign it later on. Moreover, please be sure to only put your written answers in the provided cells. 

**Deadline:**

This assignment is due Wednesday, February 9th at 12:00 PM (before the in-class midterm). Note the earlier deadline! Late work will penalized as stated in the syllabus.

Directly sharing answers is not okay, but discussing problems with the course staff or with other students is encouraged.

You should start early so that you have time to get help if you're stuck. 

First, set up the notebook by running the cell below.

In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

import warnings
warnings.simplefilter('ignore', FutureWarning)

The webpage https://opendata-townofchapelhill.hub.arcgis.com/ was created to increase government transparency by facilitating public access to local government information. With this web-based service, anyone in the community or around the world can access an ever-growing catalog of datasets from Town of Chapel Hill departments. Users can create graphs, charts, and maps based on the datasets, as well as download data, interact with it, and reuse it.

## 1. Chapel Hill Arrest Records

 The `Police_Arrests_Made` dataset contains arrests made by the Chapel Hill Police Department. Multiple individuals may have been arrested under the same arrest number during related incidents or arrests. Data is available since January 1, 2010 and updated quarterly. The table contains information for arrests in Chapel Hill for those aged 18 and above. Run the cell below to import the datset.


In [ ]:
Arrests = Table.read_table('Police_Arrests_Made.csv')
Arrests.show(5)

**Question 1.1.** Construct a table `Primary_Charges` with the columns `Primary_Charge` and `Number_of_Charges` that contains the 10 most seen primary charges in the data, as well as how many times each of those primary charges occurred in the data.

In [ ]:
Primary_Charges = ...
Primary_Charges

**Question 1.2.** Use the `Primary_Charges` table to construct a bar chart for these 10 most seen primary charges (in descending order by frequency).

**Question 1.3.** Alcohol use seems to be prevelent in many of the 10 most seen charges. For the arrests with a primary charge of `OPEN CONTAINER`, construct a histogram for the ages of those arrested. Your histogram should begin at the age of 18, have bins 3 years wide, and range over all possible ages in the data.

**Question 1.4.** What can you learn from this histogram? Specifically describe two features of the histogram and discuss what age groups seem to be most often arrested with open container as the primary charge.

*Write your answer here, replacing this text.*

**Question 1.5.** Is there a relationship between the average age for arrests depending on the type of arrest and if drugs or alcohol are present? 

To answer this question, make a table, `Arrest_types_drugs_or_alcohol`,  containing types of arrest as the rows, and the presence of drugs or alcohol as the columns. The values in the rows should correspond to the average age of those arrested for each combination of categories. Remove the rows with the `Drugs_or_Alcohol_Present` values of nan (blank) or *U* (unknown) as well as the column with the *Type_of_Arrest* value nan (blank).  

Hint: Run the cell below with the table `Arrest_NoBlanks` and use this table to construct the required table for this question. In the Arrests table there are many blank values for the age of those arrested. These blanks may lead to issues when attempting this question. Also dropped is the *Y* column for lattitude, as it may cause confusion for some possible outputs.

Note: Depending on how you go about answering this question, you may encounter a red box appearing below your code cell with a warning message. Don't worry too much about the message. Warnings are raised by Python when it encounters an unusual condition in your code, but the condition is not severe enough to warrant throwing an error.

In [ ]:
Arrests_NoBlanks = Table.read_table('Police_Arrests_Made.csv').where("Age", are.above_or_equal_to(18)).drop('Y')

In [ ]:
Arrest_types_drugs_or_alcohol = ...
Arrest_types_drugs_or_alcohol

**Question 1.6.** What does the `Arrest_types_drugs_or_alcohol` table tell you about the relationship between the average age for arrests depending on the type of arrest and if drugs or alcohol are present?

*Write your answer here, replacing this text.*

## 2. Bicycle Crashes in Chapel Hill

Dr. McLean lives not far from campus and is considering commuting from home to Hanes Hall on his bicycle. First he decicdes that he needs to investigate if it's safe to ride in Chapel Hill. Chapel Hill Open data provides a data set of all bicycle crashes reported in North Carolina (12,173 of them!) that occurred between 2007 and 2019. Begin by importing the dataset in the cell below.

In [ ]:
Bike = bike = Table.read_table('NCDOT_BikePedCrash.csv')
bike.show(5)

**Question 2.1** Construct a new table `Bike_CH` that contains only the crashes that occurred in `Chapel Hill`. 

In [ ]:
Bike_CH = ...
Bike_CH

**Question 2.2** Use the `Bike_CH` table to determine the average number of reported bicycle crashes per year in Chapel Hill and assign this value to `ave_per_year`.

In [ ]:
ave_per_year = ...
ave_per_year

Of the reported bicycle crashes that occurred in Chapel Hill, what proportion of them occurred within 0.5 miles of Dr. McLean's office in Hanes Hall? Answering this question will take a few steps. Since the longitude (`X`) and latitude (`Y`) of each bicycle crash is listed in the table, we can use those to determine a distance from Hanes Hall. Googlemaps reports that the longitude and lattide of Hanes Hall is (-79.0513, 35.9107). To convert the differences in these latitudes and longitudes from Hanes Hall, we'll need to use Haversine's Formula.

### 2.1 Haversine Formula

It might seem straightforward to pull out a map and measure the distance between two distant points, however, the larger the distance the bigger the distortion caused by traveling on the curved surface of the Earth as opposed to flat 2D space. So while the distance you measure to your neighbouring town won't be too bad, if you're measuring between London and Rio the curvature of the Earth will make a big difference to the distance that you'll travel. To help figure out the correct distance there's the haversine formula.

The haversine formula allows you to calculate the shortest distance between two points on a sphere using their latitudes and longitudes — this will be the arc between them on the great circle that includes both points. A great circle is a circle on a sphere with the same center as the sphere, like the Equator. The haversine formula isn't perfect in practice, as Earth isn't a perfect sphere

https://sketchplanations.com/the-haversine-formula

<img src="haversine.png"/>

The formula itself is a bit complicated, so it's written for you in the cell below. The arguments are the latitude and longitudes of two points on earth, with the function returning the distance between them in kilometers. Run the cell below.

In [ ]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

**Question 2.1.1.** Use the haversine formula to determine the distance in kilometers for the first bicycle crash in the `Bike_CH` table and Hanes Hall. Assign this value to `First_Crash_Distance`. 

Do we really need to consider the curvature of the Earth when traveling locally in Chapel Hill? No ... but we're going to do it anyways!

In [ ]:
First_Crash_Distance = ...
First_Crash_Distance

**Question 2.1.2.** Next you will determine the distance of each bicycle crash in the `Bike_CH` table from Hanes Hall. Begin by creating a table `Bike_CH_with_Hanes` that contains the same rows and columns as `Bike_NC`, and adds two new columns: `X Hanes` and `Y Hanes`. These columns will have the values for the longitude of Hanes Hall (`X Hanes`) and latitude of Hanes Hall (`Y Hanes`) repeated for each row of the column. To do this, you can use the np.repeat() function from numpy. This function takes in two arguments, first the values that you want to repeat, and then how many times you want to repeat it. For instance in the cell below an array is creating by repeating 0 20 times.

In [ ]:
np.repeat(0, 20)

In [ ]:
Bike_CH_with_Hanes = ...
Bike_CH_with_Hanes

**Question 2.1.3.** Next you will use the `apply` function to determine the distances of each bicycle crash from Hanes Hall. Create a new table `Bike_CH_Dist_From_Hanes` which contains the same columns and rows as `Bike_CH_with_Hanes` as well as a new column `Distance From Hanes` which contains the values in kilometers for the distance from each bicycle accident from Hanes Hall. 

In [ ]:
Bike_CH_Dist_From_Hanes = ...
Bike_CH_Dist_From_Hanes

**Question 2.1.4** What proportion of the accidents in the `Bike_CH_with_Hanes` table are within 0.5 miles of Hanes Hall? Note that the Haversine function calculates the distance in kilometers. One mile is approximately 1.60934 kilometers. Assign this value to `Prop_within_half_mile`.

In [ ]:
Prop_within_half_mile = ...
Prop_within_half_mile

Congratulations, you're done with homework 4!